In [1]:
#CONEXÃO GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#INSTALANDO BIBLIOTECAS
!pip install pandas

In [ ]:
def carregar_dfs(pasta_familia,pasta_cargos,ano):
  df1 = pd.read_csv(f'{pasta_familia}/GeralX{ano}.csv',encoding='utf-8',sep=';',low_memory=False) #GERAL FAMILIA
  df2 = pd.read_csv(f'{pasta_cargos}/FEMX{ano}.csv',encoding='utf-8',sep=';',low_memory=False)    #GERAL CARGOS FEMININO
  df3 = pd.read_csv(f'{pasta_cargos}/MASX{ano}.csv',encoding='utf-8',sep=';',low_memory=False)    #GERAL CARGOS MASCULINO
  df1=df1[df1['Total'] == 1]
  return [df1, df2, df3]

In [ ]:
def ajuste_status(dfs):
  for i in range(len(dfs)):
   dfs[i].loc[(dfs[i]['Status'] == 'Admitido') & ((dfs[i]['Periodo'] == '{ñ class}') | (dfs[i]['Periodo'] == 'N/I') | (dfs[i]['Periodo'].isna())), 'Periodo'] = 'Não admitido ano'
   dfs[i].loc[(dfs[i]['Status'] == 'Demitido') & ((dfs[i]['Periodo'] == '{ñ class}') | (dfs[i]['Periodo'] == 'N/I') | (dfs[i]['Periodo'].isna())), 'Periodo'] = 'Não desligado ano'
   dfs[i].loc[dfs[i]['Periodo'] == 'Não admitido ano', 'Status'] = 'Admitidos ano anterior'
   dfs[i].loc[dfs[i]['Periodo'] == 'Não desligado ano', 'Status'] = 'Nao demitidos'
  return dfs

In [ ]:
def valores_nulos_nclass(dfs):
  for i in range(len(dfs)):
    dfs[i]['Faixa Etaria'] = dfs[i]['Faixa Etaria'].replace('{ñ class}', 'N/I').fillna('N/I')
    dfs[i]['Media_Remu'] = dfs[i]['Media_Remu'].replace('{ñ class}', 'N/I').fillna('N/I')
    dfs[i]['Tempo_Emprego'] = dfs[i]['Tempo_Emprego'].replace('{ñ class}', 'N/I').fillna('N/I')
  return dfs

In [ ]:
def ordena_coluna(dfs):
  for i in range(len(dfs)):
    dfs[i] = dfs[i][['sigla-uf','Periodo','Faixa Etaria', 'Media_Remu', 'Tempo_Emprego' ] +
     [col for col in dfs[i].columns if col not in ['sigla-uf','Periodo', 'Faixa Etaria', 'Media_Remu', 'Tempo_Emprego']]]
  return dfs

In [ ]:
def ordena_valores(dfs):
  for i in range(len(dfs)):
    dfs[i] = dfs[i].sort_values(by=['sigla-uf','Periodo','Faixa Etaria','Media_Remu',
                                    'Tempo_Emprego'],ascending=[True,True,True,True,True]).reset_index(drop=True)
  return dfs

In [ ]:
def ajustes_finais(df):
  df = df.astype({
    'Ano': int,
    'Regiao' : str,
    'UF': str,
    'sigla-uf': str,
    'Municipio': str,
    'Familia CBO 2002': str,
    'Sexo': str,
    'Faixa Etaria': str,
    'Periodo': str,
    'Status': str,
    'Tempo_Emprego': str,
    'Media_Remu': str,
    'Total': int,
    'Cargo CBO 2002': str,
    'Cod CBO 2002': str
  })
  df = df[['UF', 'sigla-uf', 'Regiao', 'Municipio', 'Ano', 'Periodo', 'Faixa Etaria', 'Sexo', 'Familia CBO 2002',
           'Cargo CBO 2002', 'Cod CBO 2002', 'Status', 'Tempo_Emprego', 'Media_Remu', 'Total']]
  return df

In [ ]:
def contatena_geral(dfs):
  familia = ['ANALISTA DE SISTEMAS COMPUTACIONAIS','DIRETORES DE SERVICOS DE INFORMATICA',
        'ENGENHEIROS EM COMPUTACAO', 'ESPECIALISTAS EM INFORMATICA','GERENTES DE TECNOLOGIA DA INFORMACAO',
        'TECNICOS EM OPERACAO E MONITORAMENTO DE COMPUTADORES','TECNICOS EM PROGRAMACAO'] # ALTERAR DE ACORDO COM AS FAMILIAS ESCOLHIDAS E NOMES DE ARQUIVOS SALVOS
  a = 0
  while a <2:
        if a == 0 :
          sexo = 'Masculino'
        else :
          sexo = 'Feminino'
        for f in familia:
            df_a1 =dfs[0][(dfs[0]['Status'] == 'Ativo') &(dfs[0]['Sexo'] == sexo) & (dfs[0]['Familia CBO 2002'] == f)]
            df_a2 =dfs[1][(dfs[1]['Status'] == 'Ativo') &(dfs[1]['Sexo'] == sexo) & (dfs[1]['Familia CBO 2002'] == f)]
            df_ad1 =dfs[0][(dfs[0]['Status'] == 'Admitido') &(dfs[0]['Sexo'] == sexo) & (dfs[0]['Familia CBO 2002'] == f)]
            df_ad2 =dfs[1][(dfs[1]['Status'] == 'Admitido') &(dfs[1]['Sexo'] == sexo) & (dfs[1]['Familia CBO 2002'] == f)]
            df_ada1 =dfs[0][(dfs[0]['Status'] == 'Admitidos ano anterior') &(dfs[0]['Sexo'] == sexo) & (dfs[0]['Familia CBO 2002'] == f)]
            df_ada2 =dfs[1][(dfs[1]['Status'] == 'Admitidos ano anterior') &(dfs[1]['Sexo'] == sexo) & (dfs[1]['Familia CBO 2002'] == f)]
            df_d1 =dfs[0][(dfs[0]['Status'] == 'Demitido') &(dfs[0]['Sexo'] == sexo) & (dfs[0]['Familia CBO 2002'] == f)]
            df_d2 =dfs[1][(dfs[1]['Status'] == 'Demitido') &(dfs[1]['Sexo'] == sexo) & (dfs[1]['Familia CBO 2002'] == f)]
            df_nd1 =dfs[0][(dfs[0]['Status'] == 'Nao demitidos') &(dfs[0]['Sexo'] == sexo) & (dfs[0]['Familia CBO 2002'] == f)]
            df_nd2 =dfs[1][(dfs[1]['Status'] == 'Nao demitidos') &(dfs[1]['Sexo'] == sexo) & (dfs[1]['Familia CBO 2002'] == f)]
            dfs_temp = [df_a1,df_a2,df_ad1,df_ad2, df_ada1, df_ada2,df_d1,df_d2,df_nd1,df_nd2]
            dfs_temp = ordena_coluna(dfs_temp)
            dfs_temp = ordena_valores(dfs_temp)
            df_a1,df_a2,df_ad1,df_ad2, df_ada1, df_ada2,df_d1,df_d2,df_nd1,df_nd2 = dfs_temp
            colunas_extras = [col for col in df_a1.columns if col not in df_a2.columns]
            df_ativo = pd.concat([df_a2,df_a1[colunas_extras]], axis=1)
            colunas_extras_ad = [col for col in df_ad1.columns if col not in df_ad2.columns]
            df_ad = pd.concat([df_ad2, df_ad1[colunas_extras_ad]], axis=1)
            colunas_extras_ada = [col for col in df_ada1.columns if col not in df_ada2.columns]
            df_ada = pd.concat([df_ada2, df_ada1[colunas_extras_ada]], axis=1)
            colunas_extras_d = [col for col in df_d1.columns if col not in df_d2.columns]
            df_d = pd.concat([df_d2, df_d1[colunas_extras_d]], axis=1)
            colunas_extras_nd = [col for col in df_nd1.columns if col not in df_nd2.columns]
            df_nd = pd.concat([df_nd2, df_nd1[colunas_extras_nd]], axis=1)
            if f =='ANALISTA DE SISTEMAS COMPUTACIONAIS': # ALTERAR DE ACORDO COM AS FAMILIAS ESCOLHIDAS E NOMES DE ARQUIVOS SALVOS
              df_final_temp = pd.concat([df_ativo,df_ad,df_ada,df_d,df_nd], axis=0)
              del df_ativo,df_ad,df_ada,df_d,df_nd
            else:
              df_final_temp = pd.concat([df_final_temp,df_ativo,df_ad,df_ada,df_d,df_nd], axis=0)
        if a == 0 :
          df_final = df_final_temp
          del df_final_temp
        else:
          df_final = pd.concat([df_final, df_final_temp], axis=0)
          df_final = ajustes_finais(df_final)
        a += 1
  return df_final

In [ ]:
import pandas as pd
import numpy as np

ano = 2016
pasta_familia = '/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Familias_CBO_Manipulados' #ALTERAR PASTA
pasta_cargos = '/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Cargos_CBO_Manipulados' #ALTERAR PASTA
pasta_dados = '/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Dados_Finais' #ALTERAR PASTA

while ano <= 2016:
  print(ano)
  dfs = []
  dfs2 = []
  dfs = carregar_dfs(pasta_familia,pasta_cargos,ano)
  df1, df2, df3 = dfs
  dfs = [df1, df2, df3]
  dfs = ajuste_status(dfs)
  df1, df2, df3 = dfs
  df4= pd.concat([df2,df3],axis=0)
  dfs2 = [df1, df4]
  dfs2 = valores_nulos_nclass(dfs2)
  df1,df4 = dfs2
  print('concatenando')
  df5 = contatena_geral(dfs2)
  print('salvando')
  df5.to_csv(f'{pasta_dados}/FinalX{ano}.csv',index=False,sep=';', encoding='utf-8')
  print('salvo')
  del dfs,dfs2,df1,df2,df3,df4,df5
  ano +=1

2016
concatenando
salvando
salvo


In [ ]:
import pandas as pd
import numpy as np

ano = 2015
pasta_dados = '/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Dados_Finais' #ALTERAR PASTA

while ano <= 2024:
  print(ano)
  df_temp = pd.read_csv(f'{pasta_dados}/FinalX{ano}.csv',encoding='utf-8',sep=';',low_memory=False)
  if ano == 2015:
    df_final = df_temp
  else:
    df_final = pd.concat([df_final, df_temp], axis=0)
  del df_temp
  ano +=1

print('salvando')
df_final.to_csv(f'{pasta_dados}/FinalXGeral.csv',index=False,sep=';', encoding='utf-8')
print('salvo')

2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
salvando
salvo
